In [ ]:
import tensorflow as tf
print("TensorFlow version: " + tf.__version__)

In [ ]:
TUTORIAL_NAME = 'Tutorial1'
MODEL_NAME = 'basicTFonAndroid'
SAVED_MODEL_PATH = '../' + TUTORIAL_NAME+'_Saved_model/'

In [ ]:
A = tf.placeholder(tf.float32, shape=[1], name='modelInputA') # input a
B = tf.placeholder(tf.float32, shape=[1], name='modelInputB') # input b

# A tensorflow variable not used anywhere in this model
W = tf.Variable(tf.zeros(shape=[1]), dtype=tf.float32, name='W') # weights

# sum of two matrices element-wise, in our case two numbers in 
AB = tf.add(A, B, name='modelOutputAB')

In [ ]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [ ]:
sess.run(AB,{A:[2],B:[4]})

In [ ]:
tf.train.write_graph(sess.graph_def, SAVED_MODEL_PATH , MODEL_NAME + '.pbtxt')

In [ ]:
saver = tf.train.Saver()
saver.save(sess, SAVED_MODEL_PATH + MODEL_NAME + '.ckpt')

In [ ]:
from tensorflow.python.tools import freeze_graph
from tensorflow.python.tools import optimize_for_inference_lib

In [ ]:
# Freeze the graph

input_graph = SAVED_MODEL_PATH+MODEL_NAME+'.pbtxt'
input_saver = ""
input_binary = False
input_checkpoint = SAVED_MODEL_PATH+MODEL_NAME+'.ckpt'
output_node_names = 'modelOutputAB'
restore_op_name = "save/restore_all"
filename_tensor_name = "save/Const:0"
output_graph = SAVED_MODEL_PATH+'frozen_'+MODEL_NAME+'.pb'
clear_devices = True
initializer_nodes = ""
variable_names_blacklist=""

In [ ]:
freeze_graph.freeze_graph(input_graph,
                 input_saver,
                 input_binary,
                 input_checkpoint,
                 output_node_names,
                 restore_op_name,
                 filename_tensor_name,
                 output_graph,
                 clear_devices,
                 initializer_nodes,
                 variable_names_blacklist)

In [ ]:
# Below code is generating empty optimized frozen graph, need to debug
# for time being use simple frozen graph generated above
input_node_names = ['modelInputA','modelInputB']
output_frozen_graph_name = SAVED_MODEL_PATH+'frozen_'+MODEL_NAME+'.pb'
output_optimized_graph_name = SAVED_MODEL_PATH+'optimized_'+MODEL_NAME+'.pb'

from google.protobuf import text_format

from tensorflow.core.framework import graph_pb2
from tensorflow.python.framework import dtypes
from tensorflow.python.framework import graph_io
from tensorflow.python.platform import app
from tensorflow.python.platform import gfile
from tensorflow.python.tools import optimize_for_inference_lib

input_graph_def = graph_pb2.GraphDef()

with tf.gfile.Open(output_frozen_graph_name, "rb") as f:
    data = f.read()
    input_graph_def.ParseFromString(data)

output_graph_def = optimize_for_inference_lib.optimize_for_inference(
    input_graph_def,
    ["modelInputA","modelInputB"], # an array of the input node(s)
    ['modelOutputAB'], # an array of output nodes
    tf.float32.as_datatype_enum)

In [ ]:
# Save the optimized graph
f = tf.gfile.FastGFile(output_optimized_graph_name, "w")
f.write(output_graph_def.SerializeToString())